# Introduction to Statistics in Python

## 0 - Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os, sys
import pandas as pd
from scipy.stats import binom, uniform


# Importing the course datasets
deals = pd.read_csv("datasets/amir_deals.csv")
happiness = pd.read_csv("datasets/world_happiness.csv")
food_consumption = pd.read_csv("datasets/food_consumption.csv")
restaurant_groups = pd.read_csv("datasets/restaurant_groups.csv")

es = lambda n: '=' * n

## 1 - Summary Statistics

##### Measure of Center - Mean and Median - Belgium and USA

In [ ]:
# Subset for Belgium and USA only
be_and_usa = food_consumption[(food_consumption.country == 'Belgium') | (food_consumption.country == 'USA')]

# Group by country, select consumption column, and compute mean and median
print(be_and_usa.groupby(['country'])['consumption'].agg([np.mean, np.median]))

##### Mean vs. Median

In [ ]:
# Subset for food_category equals rice
rice_consumption = food_consumption.query('food_category == "rice"')

co2_mean_median = rice_consumption.agg({'co2_emission': [np.mean, np.median]})
co2_mean = co2_mean_median.iat[0, 0]
co2_median = co2_mean_median.iat[1, 0]

# Histogram of co2_emission for rice and show plot
rice_consumption.co2_emission.hist()
plt.vlines(x = co2_mean, ymin = 0, ymax = co2_mean, colors = 'red', label = 'Mean')
plt.vlines(x = co2_median, ymin = 0, ymax = co2_mean, colors = 'green', label = 'Median')
plt.show()

##### Quartiles, Quantiles, and Quintiles

In [ ]:
# Calculate the quartiles of co2_emission; both statement produce the same results; for .linspace(...), the 3rd arg is non-inclusive
print(np.quantile(food_consumption.co2_emission, np.linspace(0, 1, 5)))
# print(np.quantile(food_consumption.co2_emission, [0, .25, .5, .75, 1]))

##### Variance and Standard Deviation

In [ ]:
print(f'{es(20)} Variance and Std. Deviation by Food Category {es(20)}')
print(food_consumption.groupby('food_category')['co2_emission'].agg([np.var, np.std]))

##### Histogram of CO<sub>2</sub> Emissions for Beef and Eggs

In [ ]:
fig, ax = plt.subplots(1, 2)

# 2 lines below filter the co2_emission *Series*
beef = food_consumption.co2_emission[food_consumption.food_category == 'beef']
eggs = food_consumption.co2_emission[food_consumption.food_category == 'eggs']

# 2 lines below filter the *DataFrame*, then accesses the co2_emission *Series*
# beef = food_consumption[food_consumption.food_category == 'beef'].co2_emission.hist()
# eggs = food_consumption[food_consumption.food_category == 'eggs'].co2_emission

# Create histogram of co2_emission for food_category 'beef'
ax[0].hist(beef)
ax[0].set_title('Beef')

# Create histogram of co2_emission for food_category 'eggs'
ax[1].hist(eggs)
ax[1].set_title('Eggs')

plt.show()

##### Finding Outliers Using IQR

In [ ]:
# Calculate total co2_emission per country: emissions_by_country
emissions_by_country = food_consumption.groupby('country')['co2_emission'].sum()

# Compute the first and third quantiles and IQR of emissions_by_country
q1 = np.quantile(emissions_by_country, 0.25)
q3 = np.quantile(emissions_by_country, 0.75)
iqr = q3 - q1

# Calculate the lower and upper cutoffs for outliers
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

# Subset emissions_by_country to find outliers
outliers = emissions_by_country[(emissions_by_country < lower) | (emissions_by_country > upper)]
print(outliers)

## 2 - Random Numbers and Probability

#### Calculating Probability

##### Get Counts by Product

In [ ]:
# Count the deals for each product
counts = amir_deals['product'].value_counts()
# produces the same results as previous line; .size() includes NULLs, .count() does not
# counts = amir_deals.groupby('product').size()

##### Calculate Probability

In [ ]:
# Calculate probability of picking a deal with each product
probs = counts / counts.sum()
print(probs)

#### Sampling

##### Sampling without Replacement

In [ ]:
# seeding allows for reproducable results
np.random.seed(24)
# Sample 5 deals without replacement
sample_without_replacement = amir_deals.sample(5)

##### Sampling with Replacement

In [ ]:
# Sample 5 deals with replacement
sample_with_replacement = amir_deals.sample(5, replace = True)

#### Probability Distributions

##### Discrete Distributions

In [ ]:
# Create probability distribution
size_dist = restaurant_groups['group_size'].value_counts() / restaurant_groups.shape[0]
# Reset index and rename columns
size_dist = size_dist.reset_index()
size_dist.columns = ['group_size', 'prob']

# Expected value
expected_value = np.sum(size_dist.group_size * size_dist.prob)

# Subset groups of size 4 or more
groups_4_or_more = size_dist[size_dist.group_size >= 4]

# Sum the probabilities of groups_4_or_more
prob_4_or_more = np.sum(groups_4_or_more.prob)
print(prob_4_or_more)

##### Binomial Distribution - Simulating a Sales Deal

In [ ]:
# Set random seed to 10
np.random.seed(10)

# Simulate 52 weeks of 3 deals
deals = binom.rvs(3, .3, size = 52)

# Print mean deals won per week; min mean == 0, max mean == 3
# mean/3 == % of deals closed on average
print(np.mean(deals))

##### Calculating Binomial Probabilities - Closing 3 of 3 Deals

In [ ]:
# Probability of closing 3 out of 3 deals
prob_3 = binom.pmf(3, 3, .3)
print(prob_3)

##### Calculating Binomial Probabilities - Closing <= 1 of 3 Deals

In [ ]:
# Probability of closing <= 1 deal out of 3 deals
prob_less_than_or_equal_1 = binom.cdf(1, 3, .3)
print(prob_less_than_or_equal_1)

##### Calculating Binomial Probabilities - Closing > 1 of 3 Deals

In [ ]:
# Probability of closing > 1 deal out of 3 deals
prob_greater_than_1 = 1 - binom.cdf(1, 3, .3)
print(prob_greater_than_1)

##### Continuous Distributions

##### Uniform Distribution - Wait Time <= 5 Minutes

In [ ]:
from scipy.stats import uniform

# Min and max wait times for back-up that happens every 30 min
min_time = 0
max_time = 30

# Calculate probability of waiting more than 5 mins
prob_less_than_than_5 = uniform.cdf(5, min_time, max_time)
print(prob_greater_than_5)

##### Uniform Distribution - Wait Time > 5 Minutes

In [ ]:
prob_greater_than_5 = 1 - uniform.cdf(5, min_time, max_time)
print(prob_greater_than_5)

##### Uniform Distribution - Wait Time Between 10 and 20 Minutes

In [ ]:
# Calculate probability of waiting 10-20 mins
prob_between_10_and_20 = uniform.cdf(20, min_time, max_time) - uniform.cdf(10, min_time, max_time)
print(prob_between_10_and_20)

##### Simulating Wait Times

In [ ]:
# Set random seed to 334
np.random.seed(334)

# Generate 1000 wait times between 0 and 30 mins
wait_times = uniform.rvs(0, 30, size=1000)

# Create a histogram of simulated times and show plot
plt.hist(wait_times)
plt.show()

## 3 - More Distributions and the Central Limit Theorem

In [1]:
from zip_util import compress_folder, decompress_folder
import os, sys
from pathlib import Path

cwd = Path(os.getcwd())
datasets = cwd.joinpath('datasets')
archive = cwd.joinpath('stats_datasets.zip')
compress_folder(datasets, archive, True)

Compressing /work/files/workspace/datasets to /work/files/workspace/stats_datasets.zip
Compressed /work/files/workspace/datasets to /work/files/workspace/stats_datasets.zip
